# YOLOv8 Installation

In [1]:
import os 
os.chdir('/home/jupyter/ee_tree_counting/Models/YOLOv8 Architecture/YOLOv8l')
import matplotlib.pyplot as plt
from ultralytics import YOLO
from IPython.display import display, Image
from PIL import Image
import pandas as pd
import numpy as np

In [4]:
# Install YOLOv8
%pip install ultralytics

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (Tesla T4, 14918MiB)
Setup complete ✅ (4 CPUs, 14.6 GB RAM, 46.6/98.2 GB disk)


# YOLOv8 Architecture

In [32]:
# Download the YOLOv8 Architecture File
!wget https://raw.githubusercontent.com/ultralytics/ultralytics/main/ultralytics/cfg/models/v8/yolov8.yaml

--2024-07-25 12:25:50--  https://raw.githubusercontent.com/ultralytics/ultralytics/main/ultralytics/cfg/models/v8/yolov8.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1889 (1.8K) [text/plain]
Saving to: ‘yolov8.yaml.1’

yolov8.yaml.1       100%[===================>]   1.84K  --.-KB/s    in 0s      

2024-07-25 12:25:50 (109 MB/s) - ‘yolov8.yaml.1’ saved [1889/1889]



<div>
  <img src="https://www.stunningvisionai.com/course/yolov8-architecture.png" height="750"/>
</div>

In [7]:
# Create YOLOv8l Architecture
!cp yolov8.yaml yolov8l.yaml

## Modified YOLOv8 Architecture for Small Objects

In [8]:
# Copy YOLOv8l Small Architecture
!cp yolov8.yaml yolov8l-small.yaml

<div>
  <img src="https://www.stunningvisionai.com/course/yolov8-architecture-modification-for-small-object.png" height="685"/>
</div>

<div>
  <img src="https://www.stunningvisionai.com/course/yolov8-architecture-for-small-object.png" height="692"/>
</div>

### Training

In [30]:
# Training Original Model
!yolo detect train model=yolov8l.yaml data="/home/jupyter/ee_tree_counting/Data/Combined Dataset YOLO No Aug/data.yaml" workers=2 batch=12 device=0 epochs=100 patience=50 name=yolov8l_normal

New https://pypi.org/project/ultralytics/8.2.64 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (Tesla T4, 14918MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8l.yaml, data=/home/jupyter/ee_tree_counting/Data/Combined Dataset YOLO No Aug/data.yaml, epochs=100, time=None, patience=50, batch=12, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=2, project=None, name=yolov8l_normal11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=Fa

# Inference on Test Set

In [ ]:
model = YOLO("/home/jupyter/ee_tree_counting/Models/YOLOv8 Architecture/YOLOv8l/runs/detect/yolov8l_normal11/weights/best.pt")  

# Creating directory to store results
test_images_dir = '/home/jupyter/ee_tree_counting/Data/Combined Dataset YOLO No Aug/test/images'
output_dir = '/home/jupyter/ee_tree_counting/Data/Combined Dataset YOLO No Aug/test/YOLOv8l_normal No Aug Inference'
os.makedirs(output_dir, exist_ok=True)

# List of all test images
test_images = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith('.jpg')]
test_images.sort()

# Run inference on all test images
results = model(test_images)  

# Visualize and save the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Save results to disk
    im_rgb.save(os.path.join(output_dir, f"results_{i}.jpg"))

# Ground Truth and MAE

In [ ]:
label_dir = '/home/jupyter/ee_tree_counting/Data/Combined Dataset YOLO No Aug/test/labels'

# List of labels
labels = [os.path.join(label_dir, img) for img in os.listdir(label_dir) if img.endswith('.txt')]
labels.sort()

for idx, label in enumerate(labels):
    file = pd.read_csv(label, delimiter=' ', header=None)
    print(f'Ground Truth {idx}: {len(file)} trees')

In [4]:
# Predictions from YOLO model
predictions = [
    89, 9, 50, 47, 47, 47, 65, 72, 72, 72, 73, 107, 103, 103, 103, 102, 96, 118, 164, 19,
    71, 71, 72, 71, 88, 88, 87, 87, 87, 80, 80, 80, 79, 131, 120, 120, 120, 120, 73, 73,
    73, 73, 76, 51, 123, 123, 123, 123, 123, 91
]

# Ground truth from labels
ground_truth = [
    86, 7, 48, 48, 48, 48, 64, 69, 69, 69,
    69, 105, 114, 114, 114, 114, 109, 97, 175, 20,
    72, 72, 72, 72, 89, 89, 89, 89, 89, 77,
    77, 77, 77, 148, 112, 112, 112, 112, 72, 72,
    72, 72, 74, 50, 130, 130, 130, 130, 130, 89
]

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Image': [f'Image {i}' for i in range(len(predictions))],
    'Prediction': predictions,
    'Ground Truth': ground_truth
})

# Display the dataframe
print(results_df)

# Calculate Mean Absolute Error (MAE)
mae = (results_df['Prediction'] - results_df['Ground Truth']).abs().mean()

print(f"Mean Absolute Error (MAE): {mae:.2f}")



       Image  Prediction  Ground Truth
0    Image 0          89            86
1    Image 1           9             7
2    Image 2          50            48
3    Image 3          47            48
4    Image 4          47            48
5    Image 5          47            48
6    Image 6          65            64
7    Image 7          72            69
8    Image 8          72            69
9    Image 9          72            69
10  Image 10          73            69
11  Image 11         107           105
12  Image 12         103           114
13  Image 13         103           114
14  Image 14         103           114
15  Image 15         102           114
16  Image 16          96           109
17  Image 17         118            97
18  Image 18         164           175
19  Image 19          19            20
20  Image 20          71            72
21  Image 21          71            72
22  Image 22          72            72
23  Image 23          71            72
24  Image 24          88 

# Inference on Satellite Imagery

In [5]:
# Load our trained YOLO model
model = YOLO("/home/jupyter/ee_tree_counting/Models/YOLOv8 Architecture/runs/detect/yolov8l_normal11/weights/best.pt")  

# Creating directory to store results
sat_images_dir = '/home/jupyter/ee_tree_counting/Sample Images'

save_dir = '/home/jupyter/ee_tree_counting/Sample Images/YOLOv8l_normal Predictions'

# List of all test images
sat_imgs = [os.path.join(sat_images_dir, img) for img in os.listdir(sat_images_dir) if img.endswith('.png') or img.endswith('.jpg') or img.endswith('.tif')]
sat_imgs.sort()

# Run inference on all test images
results = model(sat_imgs)

# Visualize and save the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Save results to disk
    im_rgb.save(os.path.join(save_dir, f"results_{i}.jpg"))


0: 640x640 82 trees, 51.9ms
1: 640x640 11 trees, 51.9ms
2: 640x640 12 trees, 51.9ms
Speed: 17.6ms preprocess, 51.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)


### Training a Modified Model (Small Objects)

In [2]:
# Training a Modified Model (Small Objects)
!yolo detect train model=yolov8l-small.yaml data="/home/jupyter/ee_tree_counting/Data/Combined Dataset YOLO No Aug/data.yaml" workers=2 batch=12 device=0 epochs=100 patience=50 name=yolov8l_small

New https://pypi.org/project/ultralytics/8.2.65 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (Tesla T4, 14918MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8l-small.yaml, data=/home/jupyter/ee_tree_counting/Data/Combined Dataset YOLO No Aug/data.yaml, epochs=100, time=None, patience=50, batch=12, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=2, project=None, name=yolov8l_small2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffe

# Inference on Test Set

In [ ]:
model = YOLO("/home/jupyter/ee_tree_counting/Models/YOLOv8 Architecture/YOLOv8l/runs/detect/yolov8l_small2/weights/best.pt")  

# Creating directory to store results
test_images_dir = '/home/jupyter/ee_tree_counting/Data/Combined Dataset YOLO No Aug/test/images'
output_dir = '/home/jupyter/ee_tree_counting/Data/Combined Dataset YOLO No Aug/test/YOLOv8l_small No Aug Inference'
os.makedirs(output_dir, exist_ok=True)

# List of all test images
test_images = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith('.jpg')]
test_images.sort()

# Run inference on all test images
results = model(test_images)  

# Visualize and save the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Save results to disk
    im_rgb.save(os.path.join(output_dir, f"results_{i}.jpg"))

# Ground Truth

In [ ]:
label_dir = '/home/jupyter/ee_tree_counting/Data/Combined Dataset YOLO No Aug/test/labels'

# List of labels
labels = [os.path.join(label_dir, img) for img in os.listdir(label_dir) if img.endswith('.txt')]
labels.sort()

for idx, label in enumerate(labels):
    file = pd.read_csv(label, delimiter=' ', header=None)
    print(f'Ground Truth {idx}: {len(file)} trees')

In [6]:
predictions = [
    88, 5, 47, 45, 45, 45, 66, 69, 69, 69,
    69, 105, 106, 106, 106, 106, 90, 108, 169, 25,
    74, 74, 74, 74, 86, 86, 84, 84, 84, 78,
    78, 78, 75, 141, 123, 123, 123, 123, 69, 70,
    69, 69, 73, 50, 128, 129, 129, 128, 128, 95
]

# Ground truth from labels
ground_truth = [
    86, 7, 48, 48, 48, 48, 64, 69, 69, 69,
    69, 105, 114, 114, 114, 114, 109, 97, 175, 20,
    72, 72, 72, 72, 89, 89, 89, 89, 89, 77,
    77, 77, 77, 148, 112, 112, 112, 112, 72, 72,
    72, 72, 74, 50, 130, 130, 130, 130, 130, 89
]

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Image': [f'Image {i}' for i in range(len(predictions))],
    'Prediction': predictions,
    'Ground Truth': ground_truth
})

# Display the dataframe
print(results_df)

# Calculate Mean Absolute Error (MAE)
mae = (results_df['Prediction'] - results_df['Ground Truth']).abs().mean()

print(f"Mean Absolute Error (MAE): {mae:.2f}")

       Image  Prediction  Ground Truth
0    Image 0          88            86
1    Image 1           5             7
2    Image 2          47            48
3    Image 3          45            48
4    Image 4          45            48
5    Image 5          45            48
6    Image 6          66            64
7    Image 7          69            69
8    Image 8          69            69
9    Image 9          69            69
10  Image 10          69            69
11  Image 11         105           105
12  Image 12         106           114
13  Image 13         106           114
14  Image 14         106           114
15  Image 15         106           114
16  Image 16          90           109
17  Image 17         108            97
18  Image 18         169           175
19  Image 19          25            20
20  Image 20          74            72
21  Image 21          74            72
22  Image 22          74            72
23  Image 23          74            72
24  Image 24          86 

### Speed

Download the video

In [7]:
import cv2
import random
import time
import argparse
import os
from ultralytics import YOLO

def get_name(file_path):
  name_idx = 0
  file_pos = (file_path).rfind('\\')

  if(file_pos == -1):
      file_pos = (file_path).rfind('/')

      if(file_pos == -1):
          file_pos = 0

  name_idx = file_pos + 1

  name = file_path[name_idx:]

  return name

def get_save_path(file_name, folder_name):
  path = "result"
  save_path = os.path.join(path, folder_name)

  exists = os.path.exists(save_path)

  if(not exists):
      os.makedirs(save_path)

  save_path = os.path.join(save_path, file_name)

  return save_path

def draw_box(img, result, class_list, colors, label_size) :
  # Get information from result
  xyxy = result.boxes.xyxy.numpy()
  confidence = result.boxes.conf.numpy()
  class_id = result.boxes.cls.numpy().astype(int)
  # Pack together for easy use
  sum_output = list(zip(class_id, confidence, xyxy))
  # Copy image, in case that we need original image for something
  out_image = img.copy()

  for run_output in sum_output :
    # Unpack
    label, con, box = run_output
    # Choose color
    box_color = colors[int(label)]
    text_color = (255,255,255)
    # Get Class Name
    label = class_list[int(label)]
    # Draw object box
    first_half_box = (int(box[0]),int(box[1]))
    second_half_box = (int(box[2]),int(box[3]))
    cv2.rectangle(out_image, first_half_box, second_half_box, box_color, 2)
    # Create text
    text_print = '{label} {con:.2f}'.format(label = label, con = con)
    # Locate text position
    text_location = (int(box[0]), int(box[1] - 10 ))
    # Get size and baseline
    labelSize, baseLine = cv2.getTextSize(text_print, cv2.FONT_HERSHEY_SIMPLEX, label_size, 1)

    # Draw text's background
    cv2.rectangle(out_image
                    , (int(box[0]), int(box[1] - labelSize[1] - 10 ))
                    , (int(box[0])+labelSize[0], int(box[1] + baseLine-10))
                    , box_color , cv2.FILLED)
    # Put text
    cv2.putText(out_image, text_print ,text_location
                , cv2.FONT_HERSHEY_SIMPLEX , label_size
                , text_color, 2, cv2.LINE_AA)

  return out_image

def draw_fps(avg_fps, combined_img):
  avg_fps_str = float("{:.2f}".format(avg_fps))

  cv2.rectangle(combined_img, (10,2), (280,50), (255,255,255), -1)
  cv2.putText(combined_img, "FPS: "+str(avg_fps_str), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), thickness=3)

  return combined_img

In [11]:
import cv2
import random
import time
import os
from ultralytics import YOLO

def get_name(file_path):
    name_idx = 0
    file_pos = file_path.rfind('\\')

    if file_pos == -1:
        file_pos = file_path.rfind('/')

        if file_pos == -1:
            file_pos = 0

    name_idx = file_pos + 1
    name = file_path[name_idx:]

    return name

def get_save_path(file_name, folder_name):
    path = "result"
    save_path = os.path.join(path, folder_name)

    exists = os.path.exists(save_path)

    if not exists:
        os.makedirs(save_path)

    save_path = os.path.join(save_path, file_name)

    return save_path

def draw_box(img, result, class_list, colors, label_size):
    xyxy = result.boxes.xyxy.numpy()
    confidence = result.boxes.conf.numpy()
    class_id = result.boxes.cls.numpy().astype(int)
    sum_output = list(zip(class_id, confidence, xyxy))
    out_image = img.copy()

    for run_output in sum_output:
        label, con, box = run_output
        box_color = colors[int(label)]
        text_color = (255, 255, 255)
        label = class_list[int(label)]
        first_half_box = (int(box[0]), int(box[1]))
        second_half_box = (int(box[2]), int(box[3]))
        cv2.rectangle(out_image, first_half_box, second_half_box, box_color, 2)
        text_print = f'{label} {con:.2f}'
        text_location = (int(box[0]), int(box[1] - 10))
        labelSize, baseLine = cv2.getTextSize(text_print, cv2.FONT_HERSHEY_SIMPLEX, label_size, 1)
        cv2.rectangle(out_image, (int(box[0]), int(box[1] - labelSize[1] - 10)),
                      (int(box[0]) + labelSize[0], int(box[1] + baseLine - 10)), box_color, cv2.FILLED)
        cv2.putText(out_image, text_print, text_location, cv2.FONT_HERSHEY_SIMPLEX, label_size, text_color, 2, cv2.LINE_AA)

    return out_image

def draw_fps(avg_fps, combined_img):
    avg_fps_str = float("{:.2f}".format(avg_fps))
    cv2.rectangle(combined_img, (10, 2), (280, 50), (255, 255, 255), -1)
    cv2.putText(combined_img, "FPS: " + str(avg_fps_str), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), thickness=3)
    return combined_img

def detection(source, model, folder_name, img_size, label_size=1):
    cap = cv2.VideoCapture(source)
    model_path = model
    yolov8_detector = YOLO(model_path)

    label_map = yolov8_detector.names
    COLORS = [[random.randint(0, 255) for _ in range(3)] for _ in label_map]

    frame_count = 0
    total_fps = 0
    avg_fps = 0

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    video_frames = []

    while cap.isOpened():
        try:
            ret, frame = cap.read()
            if not ret:
                break
        except Exception as e:
            print(e)
            continue

        start = time.time()
        results = yolov8_detector.predict(frame, imgsz=img_size, verbose=False)
        result = results[0].cpu()

        combined_img = draw_box(frame, result, label_map, COLORS, label_size)
        end = time.time()

        frame_count += 1
        fps = 1 / (end - start)
        total_fps += fps
        avg_fps = total_fps / frame_count

        combined_img = draw_fps(avg_fps, combined_img)
        video_frames.append(combined_img)

        print(f"({frame_count:2d} / {total_frames:2d}) Frames Processed")

    print("\nCreate a Video:")

    file_name = get_name(source)
    save_path = get_save_path(file_name, folder_name)
    out = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'XVID'), int(avg_fps), (frame_width, frame_height))

    for frame in video_frames:
        out.write(frame)

    out.release()
    cap.release()

    print("Video is saved in: " + save_path)


In [ ]:
detection("/home/jupyter/ee_tree_counting/Data/Combined Dataset YOLO No Aug/test/YOLOv8l_small Video Inference/Tree Video.mp4", 
          "/home/jupyter/ee_tree_counting/Models/YOLOv8 Architecture/YOLOv8l/runs/detect/yolov8l_small2/weights/best.pt", 
          "yolov8l-small", 640)

## Modified YOLOv8 Architecture for Medium Objects

<div>
  <img src="https://www.stunningvisionai.com/course/yolov8-architecture-modification-for-medium-object.png" height="685"/>
</div>

<div>
  <img src="https://www.stunningvisionai.com/course/yolov8-architecture-for-medium-object.png" height="685"/>
</div>

### Training a Modified Model (Medium Objects)

In [ ]:
# Training a Modified Model (Medium Objects)
!yolo detect train model=yolov8l-medium.yaml data="/home/jupyter/ee_tree_counting/Data/Combined Dataset YOLO No Aug/data.yaml" workers=2 batch=12 device=0 epochs=100 patience=50 name=yolov8_sperm_medium

## Modified YOLOv8 Architecture for Big Objects

In [ ]:
# Copy YOLOv8l Big Architecture
!cp yolov8.yaml yolov8l-big.yaml

<div>
  <img src="https://www.stunningvisionai.com/course/yolov8-architecture-modification-for-big-object.png" height="685"/>
</div>

<div>
  <img src="https://www.stunningvisionai.com/course/yolov8-architecture-for-big-object.png" height="720"/>
</div>

### Download the dataset

In [ ]:
# Go to the data folder
%cd data

In [ ]:
# Download the dataset
!gdown https://drive.google.com/uc?id=1FW0QR0jITbhLiHL_B5HqYrowQtoQukFX

In [ ]:
# Unzip Dataset
!unzip tomato_leaf_dataset.zip

### Download the data file

In [ ]:
# Download the data file
!gdown https://drive.google.com/uc?id=1Q50HdUtbLscs03I2HFm5r4Ah0gWHS5Kd

In [ ]:
# Go to the root folder
%cd ../

### Training

In [ ]:
# Training Original Model
!yolo detect train model=yolov8l.yaml data=data/tomato_leaf.yaml workers=2 batch=12 device=0 epochs=100 patience=50 name=yolov8_tomato_leaf

### Training a Modified Model (Big Objects)

In [ ]:
# Training a Modified Model (Big Objects)
!yolo detect train model=yolov8l-big.yaml data=data/tomato_leaf.yaml workers=2 batch=12 device=0 epochs=100 patience=50 name=yolov8_tomato_leaf_big

### Speed

Download the image

In [ ]:
# Download the image
!gdown https://drive.google.com/uc?id=1ivZMXvi3vUFQQXPdZdkAyYH9Q8LYUjdp

In [ ]:
# Move the image to inference folder
!mv tomato_healthy.JPG inference/

Inference

In [ ]:
# Original YOLOv8 model
!yolo detect predict model=runs/detect/yolov8_tomato_leaf/weights/best.pt source=inference/tomato_healthy.JPG save=True

Show detection result

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def show_image(image_path):
  img = mpimg.imread(image_path)
  plt.imshow(img)
  plt.axis('off')
  plt.show()

In [ ]:
# Original YOLOv8 model
show_image("runs/detect/predict/tomato_healthy.JPG")

In [ ]:
# Modified YOLOv8 model
show_image("runs/detect/predict2/tomato_healthy.JPG")

# YOLOv8 for big objects with TensorRT

In [ ]:
!pip install tensorrt tensorrt_lean tensorrt_dispatch

In [ ]:
!pip install onnx onnxsim onnxruntime-gpu

## Convert the Model


In [ ]:
# Export YOLOv8 Model to Tensorrt
!yolo export model=runs/detect/yolov8_tomato_leaf_big/weights/best.pt format=engine half=True device=0

## mAP

In [ ]:
# Original YOLOv8 model
!yolo detect val model=runs/detect/yolov8_tomato_leaf/weights/best.pt data=data/tomato_leaf.yaml iou=0.5 name=yolov8_tomato_leaf_val

In [ ]:
# YOLOv8 for big objects with TensorRT
!yolo detect val model=runs/detect/yolov8_tomato_leaf_big/weights/best.engine data=data/tomato_leaf.yaml iou=0.5 name=yolov8_tomato_leaf_big_tensorrt_val

## Speed

In [ ]:
# Original YOLOv8 model
!yolo detect predict model=runs/detect/yolov8_tomato_leaf/weights/best.pt source=inference/tomato_healthy.JPG save=True

In [ ]:
# YOLOv8 for big objects with TensorRT
!yolo detect predict model=runs/detect/yolov8_tomato_leaf_big/weights/best.engine source=inference/tomato_healthy.JPG save=True